In [1]:
from pycaret.time_series import *


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import gc



In [3]:
usecols = ['game_id', 'user_id', 'bet_id', 'game_type', 'odds', 'fiat_bet_amount',
       'fiat_profit_amount','fiat_is_valuable']


In [4]:
bets = pd.read_csv("data/bets.csv", usecols=usecols)


In [5]:
bets.head()


,game_id,user_id,bet_id,game_type,odds,fiat_is_valuable,fiat_bet_amount,fiat_profit_amount
0,2828375,173570,210854317,normal,0.0,t,0.048738,-0.048738
1,2828375,1339924,12148553,yellow,0.0,f,NaN,NaN
2,2828375,1139219,210854260,normal,2.0,t,0.000088,0.000088
3,2828375,1329183,12148544,yellow,0.0,t,0.188109,-0.188109
4,2828375,1378328,210854292,normal,0.0,t,0.000005,-0.000005


In [6]:
usecols = ['game_id', 'max_rate', 'total_bet_amount', 'total_win_amount',
       'total_profit_amount','player_count','bet_count','timestamp']

games = pd.read_csv("data/games.csv", usecols=usecols)
 

In [7]:

games.head()


,game_id,max_rate,total_bet_amount,total_win_amount,total_profit_amount,player_count,bet_count,timestamp
0,2828375,6.43,15.696372,26.377635,10.681263,206,217,1599622213
1,2828376,2.10,69.101741,71.544642,2.442901,294,314,1599622213
2,2828377,3.11,41.762966,32.070902,-9.692064,289,306,1599622213
3,2828378,24.97,73.392453,110.032057,36.639604,281,299,1599622213
4,2828379,1.04,58.377711,71.760673,13.382962,281,297,1599622213


In [8]:
df = pd.merge(bets, games, on='game_id', how='left')


In [9]:
df.head()


,game_id,user_id,bet_id,game_type,odds,fiat_is_valuable,fiat_bet_amount,fiat_profit_amount,max_rate,total_bet_amount,total_win_amount,total_profit_amount,player_count,bet_count,timestamp
0,2828375,173570,210854317,normal,0.0,t,0.048738,-0.048738,6.43,15.696372,26.377635,10.681263,206,217,1599622213
1,2828375,1339924,12148553,yellow,0.0,f,NaN,NaN,6.43,15.696372,26.377635,10.681263,206,217,1599622213
2,2828375,1139219,210854260,normal,2.0,t,0.000088,0.000088,6.43,15.696372,26.377635,10.681263,206,217,1599622213
3,2828375,1329183,12148544,yellow,0.0,t,0.188109,-0.188109,6.43,15.696372,26.377635,10.681263,206,217,1599622213
4,2828375,1378328,210854292,normal,0.0,t,0.000005,-0.000005,6.43,15.696372,26.377635,10.681263,206,217,1599622213


In [13]:
df = df[df['fiat_is_valuable'] == 't']
df.head()


,game_id,user_id,bet_id,game_type,odds,fiat_is_valuable,fiat_bet_amount,fiat_profit_amount,max_rate,total_bet_amount,total_win_amount,total_profit_amount,player_count,bet_count,timestamp
0,2828375,173570,210854317,normal,0.0,t,0.048738,-0.048738,6.43,15.696372,26.377635,10.681263,206,217,1599622213
2,2828375,1139219,210854260,normal,2.0,t,0.000088,0.000088,6.43,15.696372,26.377635,10.681263,206,217,1599622213
3,2828375,1329183,12148544,yellow,0.0,t,0.188109,-0.188109,6.43,15.696372,26.377635,10.681263,206,217,1599622213
4,2828375,1378328,210854292,normal,0.0,t,0.000005,-0.000005,6.43,15.696372,26.377635,10.681263,206,217,1599622213
5,2828375,1420292,210854342,normal,3.0,t,0.000107,0.000213,6.43,15.696372,26.377635,10.681263,206,217,1599622213


In [14]:
df.columns


Index(['game_id', 'user_id', 'bet_id', 'game_type', 'odds', 'fiat_is_valuable',
       'fiat_bet_amount', 'fiat_profit_amount', 'max_rate', 'total_bet_amount',
       'total_win_amount', 'total_profit_amount', 'player_count', 'bet_count',
       'timestamp'],
      dtype='object')

In [15]:
df = df.drop(columns=['fiat_is_valuable'])


In [16]:
df.columns


Index(['game_id', 'user_id', 'bet_id', 'game_type', 'odds', 'fiat_bet_amount',
       'fiat_profit_amount', 'max_rate', 'total_bet_amount',
       'total_win_amount', 'total_profit_amount', 'player_count', 'bet_count',
       'timestamp'],
      dtype='object')

In [17]:
df.head(20)


,game_id,user_id,bet_id,game_type,odds,fiat_bet_amount,fiat_profit_amount,max_rate,total_bet_amount,total_win_amount,total_profit_amount,player_count,bet_count,timestamp
0,2828375,173570,210854317,normal,0.00,0.048738,-0.048738,6.43,15.696372,26.377635,10.681263,206,217,1599622213
2,2828375,1139219,210854260,normal,2.00,0.000088,0.000088,6.43,15.696372,26.377635,10.681263,206,217,1599622213
3,2828375,1329183,12148544,yellow,0.00,0.188109,-0.188109,6.43,15.696372,26.377635,10.681263,206,217,1599622213
4,2828375,1378328,210854292,normal,0.00,0.000005,-0.000005,6.43,15.696372,26.377635,10.681263,206,217,1599622213
5,2828375,1420292,210854342,normal,3.00,0.000107,0.000213,6.43,15.696372,26.377635,10.681263,206,217,1599622213
6,2828375,1205760,12148533,red,0.00,0.000176,-0.000176,6.43,15.696372,26.377635,10.681263,206,217,1599622213
7,2828375,1335815,210854336,normal,2.00,0.000008,0.000008,6.43,15.696372,26.377635,10.681263,206,217,1599622213
8,2828375,1189902,210854446,normal,5.00,0.000283,0.001131,6.43,15.696372,26.377635,10.681263,206,217,1599622213
9,2828375,1378313,210854326,normal,2.00,0.000301,0.000301,6.43,15.696372,26.377635,10.681263,206,217,1599622213
10,2828375,1287181,210854358,normal,2.00,0.000019,0.000019,6.43,15.696372,26.377635,10.681263,206,217,1599622213


In [19]:
df['saldo'] = df.groupby('user_id')['fiat_profit_amount'].cumsum()
df['n_aposta'] = df.groupby('user_id').cumcount() + 1


In [20]:
# df[df['game_type']=='red'].sort_values('odds')

#green - 2
#yellow - 10


In [21]:
conditions = [
    (df['game_type'] == 'normal') & (df['odds'] == 0),
    (df['game_type'] == 'green'),
    (df['game_type'] == 'red'),
    (df['game_type'] == 'yellow')
]

values = [
    df['max_rate'],
    2,
    1.96,
    10
]

df['odds_adjusted'] = np.select(conditions, values, default=df['odds'])


In [22]:
df.head()


,game_id,user_id,bet_id,game_type,odds,fiat_bet_amount,fiat_profit_amount,max_rate,total_bet_amount,total_win_amount,total_profit_amount,player_count,bet_count,timestamp,saldo,n_aposta,odds_adjusted
0,2828375,173570,210854317,normal,0.0,0.048738,-0.048738,6.43,15.696372,26.377635,10.681263,206,217,1599622213,-0.048738,1,6.43
2,2828375,1139219,210854260,normal,2.0,0.000088,0.000088,6.43,15.696372,26.377635,10.681263,206,217,1599622213,0.000088,1,2.00
3,2828375,1329183,12148544,yellow,0.0,0.188109,-0.188109,6.43,15.696372,26.377635,10.681263,206,217,1599622213,-0.188109,1,10.00
4,2828375,1378328,210854292,normal,0.0,0.000005,-0.000005,6.43,15.696372,26.377635,10.681263,206,217,1599622213,-0.000005,1,6.43
5,2828375,1420292,210854342,normal,3.0,0.000107,0.000213,6.43,15.696372,26.377635,10.681263,206,217,1599622213,0.000213,1,3.00


In [23]:

df['bet_mean'] = df.groupby('user_id')['fiat_bet_amount'].transform('mean')



In [24]:
df.head()


,game_id,user_id,bet_id,game_type,odds,fiat_bet_amount,fiat_profit_amount,max_rate,total_bet_amount,total_win_amount,total_profit_amount,player_count,bet_count,timestamp,saldo,n_aposta,odds_adjusted,bet_mean
0,2828375,173570,210854317,normal,0.0,0.048738,-0.048738,6.43,15.696372,26.377635,10.681263,206,217,1599622213,-0.048738,1,6.43,0.053502
2,2828375,1139219,210854260,normal,2.0,0.000088,0.000088,6.43,15.696372,26.377635,10.681263,206,217,1599622213,0.000088,1,2.00,0.000143
3,2828375,1329183,12148544,yellow,0.0,0.188109,-0.188109,6.43,15.696372,26.377635,10.681263,206,217,1599622213,-0.188109,1,10.00,0.467819
4,2828375,1378328,210854292,normal,0.0,0.000005,-0.000005,6.43,15.696372,26.377635,10.681263,206,217,1599622213,-0.000005,1,6.43,0.000947
5,2828375,1420292,210854342,normal,3.0,0.000107,0.000213,6.43,15.696372,26.377635,10.681263,206,217,1599622213,0.000213,1,3.00,0.000291


In [25]:
df['bet_proportion'] = df['fiat_bet_amount'] / df['bet_mean']
df.head()


,game_id,user_id,bet_id,game_type,odds,fiat_bet_amount,fiat_profit_amount,max_rate,total_bet_amount,total_win_amount,total_profit_amount,player_count,bet_count,timestamp,saldo,n_aposta,odds_adjusted,bet_mean,bet_proportion
0,2828375,173570,210854317,normal,0.0,0.048738,-0.048738,6.43,15.696372,26.377635,10.681263,206,217,1599622213,-0.048738,1,6.43,0.053502,0.910945
2,2828375,1139219,210854260,normal,2.0,0.000088,0.000088,6.43,15.696372,26.377635,10.681263,206,217,1599622213,0.000088,1,2.00,0.000143,0.611533
3,2828375,1329183,12148544,yellow,0.0,0.188109,-0.188109,6.43,15.696372,26.377635,10.681263,206,217,1599622213,-0.188109,1,10.00,0.467819,0.402098
4,2828375,1378328,210854292,normal,0.0,0.000005,-0.000005,6.43,15.696372,26.377635,10.681263,206,217,1599622213,-0.000005,1,6.43,0.000947,0.005158
5,2828375,1420292,210854342,normal,3.0,0.000107,0.000213,6.43,15.696372,26.377635,10.681263,206,217,1599622213,0.000213,1,3.00,0.000291,0.366326


In [26]:
df['risk'] = df['odds_adjusted'] * df['bet_proportion']
df.head()


,game_id,user_id,bet_id,game_type,odds,fiat_bet_amount,fiat_profit_amount,max_rate,total_bet_amount,total_win_amount,total_profit_amount,player_count,bet_count,timestamp,saldo,n_aposta,odds_adjusted,bet_mean,bet_proportion,risk
0,2828375,173570,210854317,normal,0.0,0.048738,-0.048738,6.43,15.696372,26.377635,10.681263,206,217,1599622213,-0.048738,1,6.43,0.053502,0.910945,5.857378
2,2828375,1139219,210854260,normal,2.0,0.000088,0.000088,6.43,15.696372,26.377635,10.681263,206,217,1599622213,0.000088,1,2.00,0.000143,0.611533,1.223067
3,2828375,1329183,12148544,yellow,0.0,0.188109,-0.188109,6.43,15.696372,26.377635,10.681263,206,217,1599622213,-0.188109,1,10.00,0.467819,0.402098,4.020981
4,2828375,1378328,210854292,normal,0.0,0.000005,-0.000005,6.43,15.696372,26.377635,10.681263,206,217,1599622213,-0.000005,1,6.43,0.000947,0.005158,0.033166
5,2828375,1420292,210854342,normal,3.0,0.000107,0.000213,6.43,15.696372,26.377635,10.681263,206,217,1599622213,0.000213,1,3.00,0.000291,0.366326,1.098979


In [27]:
df['risk_log'] = np.log1p(df['risk'])
df.head()


,game_id,user_id,bet_id,game_type,odds,fiat_bet_amount,fiat_profit_amount,max_rate,total_bet_amount,total_win_amount,...,player_count,bet_count,timestamp,saldo,n_aposta,odds_adjusted,bet_mean,bet_proportion,risk,risk_log
0,2828375,173570,210854317,normal,0.0,0.048738,-0.048738,6.43,15.696372,26.377635,...,206,217,1599622213,-0.048738,1,6.43,0.053502,0.910945,5.857378,1.925325
2,2828375,1139219,210854260,normal,2.0,0.000088,0.000088,6.43,15.696372,26.377635,...,206,217,1599622213,0.000088,1,2.00,0.000143,0.611533,1.223067,0.798888
3,2828375,1329183,12148544,yellow,0.0,0.188109,-0.188109,6.43,15.696372,26.377635,...,206,217,1599622213,-0.188109,1,10.00,0.467819,0.402098,4.020981,1.613625
4,2828375,1378328,210854292,normal,0.0,0.000005,-0.000005,6.43,15.696372,26.377635,...,206,217,1599622213,-0.000005,1,6.43,0.000947,0.005158,0.033166,0.032628
5,2828375,1420292,210854342,normal,3.0,0.000107,0.000213,6.43,15.696372,26.377635,...,206,217,1599622213,0.000213,1,3.00,0.000291,0.366326,1.098979,0.741451


In [ ]:
def downcast_df(df: pd.DataFrame) -> pd.DataFrame:
    float_cols = df.select_dtypes(include=['float64','float32']).columns
    df[float_cols] = df[float_cols].astype('float32', copy=False)

    int_cols = df.select_dtypes(include=['int64','int32','int']).columns
    for c in int_cols:
        if df[c].min() >= 0 and df[c].max() <= np.iinfo(np.uint32).max:
            df[c] = df[c].astype('uint32', copy=False)
        else:
            df[c] = df[c].astype('int32', copy=False)


    if 'user_id' in df.columns:
        df['user_id'] = df['user_id'].astype('category')


    if 'game_id' in df.columns:
        df['game_id'] = df['game_id'].astype('category')

    gc.collect()
    return df


In [44]:
dc_df = downcast_df(df)



In [45]:
df = dc_df.copy()


In [46]:
df = df.drop(columns=['game_type','odds','max_rate','odds_adjusted','bet_mean','bet_proportion','risk'])
df.head()


,game_id,user_id,bet_id,fiat_bet_amount,fiat_profit_amount,total_bet_amount,total_win_amount,total_profit_amount,player_count,bet_count,timestamp,saldo,n_aposta,risk_log
0,2828375,173570,210854317,0.048738,-0.048738,15.696372,26.377634,10.681263,206,217,1599622213,-0.048738,1,1.925325
2,2828375,1139219,210854260,0.000088,0.000088,15.696372,26.377634,10.681263,206,217,1599622213,0.000088,1,0.798888
3,2828375,1329183,12148544,0.188109,-0.188109,15.696372,26.377634,10.681263,206,217,1599622213,-0.188109,1,1.613625
4,2828375,1378328,210854292,0.000005,-0.000005,15.696372,26.377634,10.681263,206,217,1599622213,-0.000005,1,0.032628
5,2828375,1420292,210854342,0.000107,0.000213,15.696372,26.377634,10.681263,206,217,1599622213,0.000213,1,0.741451


In [47]:
df['avg_game_bet_amount'] = df.groupby('game_id', sort=False)['fiat_bet_amount'].transform('mean')
df['median_game_bet_amount'] = df.groupby('game_id', sort=False)['fiat_bet_amount'].transform('median')
df['avg_game_win_amount'] = df.groupby('game_id', sort=False)['fiat_profit_amount'].transform('mean')
df['median_game_win_amount'] = df.groupby('game_id', sort=False)['fiat_profit_amount'].transform('median')


In [48]:
df = df.drop(columns=['total_bet_amount','total_win_amount','total_profit_amount','player_count','bet_count'])
df.head()


,game_id,user_id,bet_id,fiat_bet_amount,fiat_profit_amount,timestamp,saldo,n_aposta,risk_log,avg_game_bet_amount,median_game_bet_amount,avg_game_win_amount,median_game_win_amount
0,2828375,173570,210854317,0.048738,-0.048738,1599622213,-0.048738,1,1.925325,0.080084,0.000151,0.054496,0.000028
2,2828375,1139219,210854260,0.000088,0.000088,1599622213,0.000088,1,0.798888,0.080084,0.000151,0.054496,0.000028
3,2828375,1329183,12148544,0.188109,-0.188109,1599622213,-0.188109,1,1.613625,0.080084,0.000151,0.054496,0.000028
4,2828375,1378328,210854292,0.000005,-0.000005,1599622213,-0.000005,1,0.032628,0.080084,0.000151,0.054496,0.000028
5,2828375,1420292,210854342,0.000107,0.000213,1599622213,0.000213,1,0.741451,0.080084,0.000151,0.054496,0.000028


In [49]:
df.head()


,game_id,user_id,bet_id,fiat_bet_amount,fiat_profit_amount,timestamp,saldo,n_aposta,risk_log,avg_game_bet_amount,median_game_bet_amount,avg_game_win_amount,median_game_win_amount
0,2828375,173570,210854317,0.048738,-0.048738,1599622213,-0.048738,1,1.925325,0.080084,0.000151,0.054496,0.000028
2,2828375,1139219,210854260,0.000088,0.000088,1599622213,0.000088,1,0.798888,0.080084,0.000151,0.054496,0.000028
3,2828375,1329183,12148544,0.188109,-0.188109,1599622213,-0.188109,1,1.613625,0.080084,0.000151,0.054496,0.000028
4,2828375,1378328,210854292,0.000005,-0.000005,1599622213,-0.000005,1,0.032628,0.080084,0.000151,0.054496,0.000028
5,2828375,1420292,210854342,0.000107,0.000213,1599622213,0.000213,1,0.741451,0.080084,0.000151,0.054496,0.000028


In [50]:
df['datetime'] = pd.to_datetime(df['timestamp'], unit='s')
df = df.sort_values(['user_id', 'datetime'])
df = df.set_index('datetime')


In [51]:
df.head()


,game_id,user_id,bet_id,fiat_bet_amount,fiat_profit_amount,timestamp,saldo,n_aposta,risk_log,avg_game_bet_amount,median_game_bet_amount,avg_game_win_amount,median_game_win_amount
datetime,,,,,,,,,,,,,
2020-09-09 04:00:03,2831698,44,212042687,0.000034,0.000001,1599624003,0.000001,1,0.000810,0.222572,0.000443,0.066997,9.498483e-06
2020-09-09 04:00:03,2831768,44,212067364,0.000034,0.000001,1599624003,0.000002,2,0.000810,0.770692,0.000152,0.214109,4.122475e-07
2020-09-09 04:00:03,2831820,44,212085405,0.000034,0.000003,1599624003,0.000005,3,0.000849,0.601125,0.000478,-0.524657,-1.838832e-04
2020-09-10 04:01:07,2832628,44,212376980,0.000035,0.000024,1599710467,0.000028,4,0.001365,0.974122,0.000277,0.700149,1.409974e-06
2020-09-10 04:01:07,2832686,44,212397916,0.000140,0.000140,1599710467,0.000169,5,0.006522,0.129546,0.000182,0.705016,9.065388e-05


In [52]:
cols = ['fiat_bet_amount', 'fiat_profit_amount','avg_game_bet_amount', 'avg_game_win_amount', 'median_game_bet_amount', 'median_game_win_amount', 'risk_log']
windows = {'2d':'2D', '7d':'7D', '14d':'14D', '21d':'21D'}

for col in cols:
    g = df.groupby('user_id')[col]
    for suf, w in windows.items():
        df[f'{col}_mean_{suf}'] = (
            g.rolling(w, min_periods=1)
             .mean()
             .reset_index(level=0, drop=True)
        )


: 

: 

In [ ]:
df.head()


,game_id,user_id,bet_id,fiat_bet_amount,fiat_profit_amount,timestamp,saldo,n_aposta,risk_log,avg_game_bet_amount,...,median_game_bet_amount_mean_14d,median_game_bet_amount_mean_21d,median_game_win_amount_mean_2d,median_game_win_amount_mean_7d,median_game_win_amount_mean_14d,median_game_win_amount_mean_21d,risk_log_mean_2d,risk_log_mean_7d,risk_log_mean_14d,risk_log_mean_21d
datetime,,,,,,,,,,,,,,,,,,,,,
2020-09-09 04:00:03,2831698,44,212042687,0.000034,0.000001,1599624003,0.000001,1,0.000810,0.222572,...,0.000443,0.000443,0.000009,0.000009,0.000009,0.000009,0.000810,0.000810,0.000810,0.000810
2020-09-09 04:00:03,2831768,44,212067364,0.000034,0.000001,1599624003,0.000002,2,0.000810,0.770692,...,0.000298,0.000298,0.000005,0.000005,0.000005,0.000005,0.000810,0.000810,0.000810,0.000810
2020-09-09 04:00:03,2831820,44,212085405,0.000034,0.000003,1599624003,0.000005,3,0.000849,0.601125,...,0.000358,0.000358,-0.000058,-0.000058,-0.000058,-0.000058,0.000823,0.000823,0.000823,0.000823
2020-09-10 04:01:07,2832628,44,212376980,0.000035,0.000024,1599710467,0.000028,4,0.001365,0.974122,...,0.000338,0.000338,-0.000043,-0.000043,-0.000043,-0.000043,0.000959,0.000959,0.000959,0.000959
2020-09-10 04:01:07,2832686,44,212397916,0.000140,0.000140,1599710467,0.000169,5,0.006522,0.129546,...,0.000306,0.000306,-0.000016,-0.000016,-0.000016,-0.000016,0.002071,0.002071,0.002071,0.002071


In [ ]:
df.sort_values(['user_id','game_id'], kind='stable', inplace=True, ignore_index=True)
df = df.set_index('game_id')


MemoryError: Unable to allocate 178. MiB for an array with shape (23335924, 1) and data type float64

In [ ]:
df.columns


Index(['game_id', 'user_id', 'bet_id', 'fiat_bet_amount', 'fiat_profit_amount',
       'saldo', 'n_aposta', 'risk_log', 'avg_game_bet_amount',
       'median_game_bet_amount', 'avg_game_win_amount',
       'median_game_win_amount', 'fiat_bet_amount_mean_2d',
       'fiat_bet_amount_mean_7d', 'fiat_bet_amount_mean_14d',
       'fiat_bet_amount_mean_21d', 'fiat_profit_amount_mean_2d',
       'fiat_profit_amount_mean_7d', 'fiat_profit_amount_mean_14d',
       'fiat_profit_amount_mean_21d', 'avg_game_bet_amount_mean_2d',
       'avg_game_bet_amount_mean_7d', 'avg_game_bet_amount_mean_14d',
       'avg_game_bet_amount_mean_21d', 'avg_game_win_amount_mean_2d',
       'avg_game_win_amount_mean_7d', 'avg_game_win_amount_mean_14d',
       'avg_game_win_amount_mean_21d', 'median_game_bet_amount_mean_2d',
       'median_game_bet_amount_mean_7d', 'median_game_bet_amount_mean_14d',
       'median_game_bet_amount_mean_21d', 'median_game_win_amount_mean_2d',
       'median_game_win_amount_mean_7d', '

In [ ]:
cols = ['fiat_bet_amount', 'fiat_profit_amount',
        'avg_game_bet_amount', 'avg_game_win_amount',
        'median_game_bet_amount', 'median_game_win_amount', 'risk_log']

windows_games = [10, 50, 100, 200]

for col in cols:
    g = df.groupby('user_id')[col]
    for w in windows_games:
        df[f'{col}_mean_{w}j'] = (
            g.rolling(window=w, min_periods=1)
             .mean()
             .reset_index(level=0, drop=True)
        )
